## Processing qua data (2022 dataset)

In [1]:
from cv2 import mean
import pandas as pd
import quaternion as Q
import numpy as np
import time
from datetime import datetime
from asyncore import write
import math
import pandas as pd
import datetime as dt

In [2]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/Participant19/qua.csv"
#print(path)
data = pd.read_csv(path)
data.head()


,id,p_id,itr_id,prj_id,ppt_id,ppt_name,hand,clt_id,w,x,y,z,timestamp
0,212444,19,119,622,16,softness,1,27,0.897418,0.059799,-0.405298,0.163701,2022-02-13 11:24:11.316
1,212445,19,119,622,16,softness,0,27,0.050747,0.424152,-0.057506,0.902337,2022-02-13 11:24:11.326
2,212446,19,119,622,16,softness,1,27,0.899481,0.057231,-0.400058,0.166167,2022-02-13 11:24:11.348
3,212447,19,119,622,16,softness,0,27,0.091317,0.417244,-0.069474,0.901522,2022-02-13 11:24:11.357
4,212448,19,119,622,16,softness,1,27,0.900094,0.055599,-0.397422,0.169693,2022-02-13 11:24:11.376


path2 = "/Users/niharawarawita/Desktop/MSc Project/Data/Lili_Experiment/Participant01/qua.csv"
#print(path)
data2 = pd.read_csv(path2)
data2.head()

data2['itr_type'].unique()

### Step 1 - Remove the ppt_name column

In [3]:
for participant in range(19,26):
    print("Participant: ", participant)
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/qua.csv"
        data = pd.read_csv(path)
        
        data.drop(['ppt_name'], axis=1, inplace=True)

        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/quat_prop.csv"
        data.to_csv(write_path, index = None)

Participant:  19
Participant:  20
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


path = "/Users/niharawarawita/Desktop/MSc Project/Data/EMG_data_collection/"+"Participant"+str(24)+"/quat_prop.csv"
#print(path)
data = pd.read_csv(path)
data.head()



### Step 2 - Sort the files in temporal order

In [4]:
for participant in range(19,26):
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/quat_prop.csv"
        data = pd.read_csv(path)
        print("Participant: ", participant)

        data['timestamp'] = pd.to_datetime(data['timestamp'])

        data.sort_values(by=['timestamp'], inplace=True)

        #itr_id = data.itr_id.unique()

        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/quat_prop_sorted.csv"
        data.to_csv(write_path,index=False)
        
        

Participant:  19
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


### Step 3 - Compute the angular velocity for the x, y and z direction

In [5]:
MSM = '%M:%S.%f'
zero = '00:00.000'
zero = datetime.strptime(zero, MSM)
labels = ['id','p_id','itr_id','prj_id','ppt_id','clt_id','hand','w','x','y','z','AVx','AVy','AVz','timestamp']

for participant in range(19,26):
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/quat_prop_sorted.csv" #"/quat_prop.csv"
        data = pd.read_csv(path)

        print("Participant: ", participant)

        data['timestamp'] = pd.to_datetime(data['timestamp'])

        hand0 = data[data['hand'] == 0]   # Create a dataset with only hand0
        hand1 = data[data['hand'] == 1]   # Create a dataset with only hand1

        time0 = [(hand0.iloc[0].at['timestamp']-zero).total_seconds()]
        time1 = [(hand1.iloc[0].at['timestamp']-zero).total_seconds()]

        quat0 = [np.quaternion(hand0.iloc[0].at['w'],hand0.iloc[0].at['x'],hand0.iloc[0].at['y'],hand0.iloc[0].at['z'])]
        quat1 = [np.quaternion(hand1.iloc[1].at['w'],hand1.iloc[1].at['x'],hand1.iloc[1].at['y'],hand1.iloc[1].at['z'])]

        ang_vel0 = []
        ang_vel1 = []

        for i in range(1,len(hand0)):
            if (hand0.iloc[i].at['itr_id'] != hand0.iloc[i-1].at['itr_id']):

                R = np.stack(quat0)

                 #t = (0,(time2-time1).total_seconds()
                t = np.stack(time0)
                vel0 = Q.quaternion_time_series.angular_velocity(R, t)
                vel0=np.stack(vel0)


                #ang_vel.append([0,0,0])
                ang_vel0.append(vel0)
                quat0 = []
                time0 = []

            quat0.append(np.quaternion(hand0.iloc[i].at['w'],hand0.iloc[i].at['x'],hand0.iloc[i].at['y'],hand0.iloc[i].at['z']))
            time0.append((hand0.iloc[i].at['timestamp']-zero).total_seconds())

        R = np.stack(quat0)
        t = np.stack(time0)
        vel0 = Q.quaternion_time_series.angular_velocity(R, t)
        vel0=np.stack(vel0)
        ang_vel0.append(vel0)


        R = []
        t = []
        prev_i = 0
        for i in range(1,len(hand1)):
            if (hand1.iloc[i].at['itr_id'] != hand1.iloc[i-1].at['itr_id']):
                R = np.stack(quat1)
                t = np.stack(time1)
                #print("len: ", (i-1)-prev_i)
                vel1 = Q.quaternion_time_series.angular_velocity(R, t)
                vel1 = np.stack(vel1)
                #ang_vel.append([0,0,0])
                #print(len(vel1))

                ang_vel1.append(vel1)

                quat1 = []
                time1 = []

            quat1.append(np.quaternion(hand1.iloc[i].at['w'],hand1.iloc[i].at['x'],hand1.iloc[i].at['y'],hand1.iloc[i].at['z']))
            time1.append((hand1.iloc[i].at['timestamp']-zero).total_seconds())
        #
        R = np.stack(quat1)
        t = np.stack(time1)
        #print("len: ", (i-1)-prev_i)
        vel1 = Q.quaternion_time_series.angular_velocity(R, t)
        vel1 = np.stack(vel1)
        ang_vel1.append(vel1)
        #print(len(ang_vel0))
        ang_vel0 = np.vstack(ang_vel0)    
        ang_vel1 = np.vstack(ang_vel1)

        #time0 = hand0['timestamp']
        #time1 = hand1['timestamp']
        tot0 = pd.DataFrame(hand0, columns=labels)
        tot1 = pd.DataFrame(hand1, columns=labels)
        tot = pd.concat([tot0, tot1])


        ang_vel = np.concatenate((ang_vel0,ang_vel1), axis=0)

        tot['AVx'] = ang_vel[:,0]
        tot['AVy'] = ang_vel[:,1]
        tot['AVz'] = ang_vel[:,2]

        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/ang_vel_quat.csv"
        tot.to_csv(write_path, index = None)



Participant:  19
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


In [6]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(23)+"/ang_vel_quat.csv"
#print(path)
data = pd.read_csv(path)
data.head()

,id,p_id,itr_id,prj_id,ppt_id,clt_id,hand,w,x,y,z,AVx,AVy,AVz,timestamp
0,382584,23,237,622,13,44,0,0.226575,0.536568,-0.218152,0.783051,-0.034368,-0.067368,0.002556,2022-02-14 11:35:31.854
1,382586,23,237,622,13,44,0,0.226596,0.536150,-0.217895,0.783402,-0.033228,-0.020500,0.013735,2022-02-14 11:35:31.877
2,382587,23,237,622,13,44,0,0.226649,0.535925,-0.217571,0.783631,-0.037174,-0.026881,0.008120,2022-02-14 11:35:31.899
3,382589,23,237,622,13,44,0,0.226717,0.535609,-0.217298,0.783903,-0.026601,-0.016786,0.013850,2022-02-14 11:35:31.921
4,382591,23,237,622,13,44,0,0.226887,0.535385,-0.216970,0.784098,-0.061637,-0.035474,-0.034565,2022-02-14 11:35:31.948


### Step 4 - Compute angular acceleration and stats

In [7]:
#labels = ['id','p_id','e_id','itr_id','itr_type','clt_id','hand','w','x','y','z','AVx','AVy','AVz','AAx','AAy','AAz','AJx','AJy','AJz','timestamp']
labels = ['id','p_id','itr_id','prj_id','ppt_id','clt_id','hand','w','x','y','z','AVx','AVy','AVz','AAx','AAy','AAz','AJx','AJy','AJz','timestamp']


for participant in range(19,26):
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/ang_vel_quat.csv"
        data = pd.read_csv(path)
        print("Participant: ", participant)

        print(type(data['timestamp'][0]))
        data['timestamp'] = pd.to_datetime(data['timestamp'])

        hand0 = data[data['hand'] == 0]
        hand1 = data[data['hand'] == 1]


        acc0x = [0]*len(hand0)
        acc1x = [0]*len(hand1)

        acc0y = [0]*len(hand0)
        acc1y = [0]*len(hand1)

        acc0z = [0]*len(hand0)
        acc1z = [0]*len(hand1)

        jerk0x = [0]*len(hand0)
        jerk1x = [0]*len(hand1)

        jerk0y = [0]*len(hand0)
        jerk1y = [0]*len(hand1)

        jerk0z = [0]*len(hand0)
        jerk1z = [0]*len(hand1)

        for i in range(1,len(hand0)):
            if hand0.iloc[i].at['itr_id'] != hand0.iloc[i-1].at['itr_id']:
                acc0x[i] = 0
                acc0y[i] = 0
                acc0z[i] = 0
                jerk0x[i] = 0
                jerk0y[i] = 0
                jerk0z[i] = 0
            else:    
                time1 = hand0.iloc[i-1]['timestamp']
                time2 = hand0.iloc[i]['timestamp']
                seconds = (time2-time1).total_seconds()
                acc_x = (hand0['AVx'].iloc[i] - hand0['AVx'].iloc[i-1])/seconds
                acc_y = (hand0['AVy'].iloc[i] - hand0['AVy'].iloc[i-1])/seconds
                acc_z = (hand0['AVz'].iloc[i] - hand0['AVz'].iloc[i-1])/seconds
                acc0x[i]=acc_x
                acc0y[i]=acc_y
                acc0z[i]=acc_z
                jerk_x = (acc0x[i] - acc0x[i-1])/seconds
                jerk_y = (acc0y[i] - acc0y[i-1])/seconds
                jerk_z = (acc0z[i] - acc0z[i-1])/seconds
                jerk0x[i]=jerk_x
                jerk0y[i]=jerk_y
                jerk0z[i]=jerk_z



        for i in range(1,len(hand1)):
            if hand1.iloc[i].at['itr_id'] != hand1.iloc[i-1].at['itr_id']:
                acc1x[i] = 0
                acc1y[i] = 0
                acc1z[i] = 0
                jerk1x[i] = 0
                jerk1y[i] = 0
                jerk1z[i] = 0
            else:
                time1 = hand1.iloc[i-1]['timestamp']
                time2 = hand1.iloc[i]['timestamp']
                seconds = (time2-time1).total_seconds()
                acc_x = (hand1['AVx'].iloc[i] - hand1['AVx'].iloc[i-1])/seconds
                acc_y = (hand1['AVy'].iloc[i] - hand1['AVy'].iloc[i-1])/seconds
                acc_z = (hand1['AVz'].iloc[i] - hand1['AVz'].iloc[i-1])/seconds
                acc1x[i]=acc_x
                acc1y[i]=acc_y
                acc1z[i]=acc_z
                jerk_x = (acc1x[i] - acc1x[i-1])/seconds
                jerk_y = (acc1y[i] - acc1y[i-1])/seconds
                jerk_z = (acc1z[i] - acc1z[i-1])/seconds
                jerk1x[i]=jerk_x
                jerk1y[i]=jerk_y
                jerk1z[i]=jerk_z

        tot0 = pd.DataFrame(hand0, columns=labels)
        tot0['AAx'] = acc0x
        tot0['AAy'] = acc0y
        tot0['AAz'] = acc0z
        tot0['AJx'] = jerk0x
        tot0['AJy'] = jerk0y
        tot0['AJz'] = jerk0z

        tot1 = pd.DataFrame(hand1, columns=labels)
        tot1['AAx'] = acc1x
        tot1['AAy'] = acc1y
        tot1['AAz'] = acc1z
        tot1['AJx'] = jerk1x
        tot1['AJy'] = jerk1y
        tot1['AJz'] = jerk1z
        #tot = pd.DataFrame([hand0['id'],hand0['p_id'],hand0['e_id'],hand0['itr_id'],hand0['itr_type'],hand0['clt_id'],hand0['hand'],
        #    hand0['x'],hand0['y'],hand0['z'],acc0x,acc0y,acc0z,hand0['timestamp']],columns=labels)

        tot3 = pd.concat([tot0,tot1])
        tot3.sort_values(by=['timestamp'],inplace=True)
        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/angularStats_quat.csv"
        tot3.to_csv(write_path, index = None)


Participant:  19
<class 'str'>
Participant:  21
<class 'str'>
Participant:  22
<class 'str'>
Participant:  23
<class 'str'>
Participant:  24
<class 'str'>
Participant:  25
<class 'str'>


In [8]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(24)+"/angularStats_quat.csv"
#print(path)
data = pd.read_csv(path)
data.head()




,id,p_id,itr_id,prj_id,ppt_id,clt_id,hand,w,x,y,...,AVx,AVy,AVz,AAx,AAy,AAz,AJx,AJy,AJz,timestamp
0,438377,24,277,622,12,54,0,0.715454,0.263440,-0.396753,...,-0.258251,-0.064080,1.183906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-14 14:44:14.858
1,438378,24,277,622,12,54,1,0.231350,-0.463977,-0.109726,...,-0.100280,0.183533,1.308658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-14 14:44:14.904
2,438379,24,277,622,12,54,0,0.717635,0.259755,-0.392300,...,-0.423856,0.123658,-0.617617,-3.523497,3.994427,-38.330261,-74.968026,84.987798,-815.537463,2022-02-14 14:44:14.905
3,438380,24,277,622,12,54,1,0.231143,-0.462888,-0.109600,...,0.065650,-0.133450,-0.834529,8.733143,-16.683320,-112.799304,459.639125,-878.069474,-5936.805461,2022-02-14 14:44:14.923
4,438381,24,277,622,12,54,0,0.723321,0.252381,-0.389067,...,-0.847606,0.060091,-0.234058,-16.949998,-2.542691,15.342358,-537.060040,-261.484688,2146.904765,2022-02-14 14:44:14.930


### Step 5 - Combine the datasets

In [9]:
path1 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(19)+"/angularStats_quat.csv"
data1 = pd.read_csv(path1)

path2 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(21)+"/angularStats_quat.csv"
data2 = pd.read_csv(path2)

path3 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(22)+"/angularStats_quat.csv"
data3 = pd.read_csv(path3)

path4 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(23)+"/angularStats_quat.csv"
data4 = pd.read_csv(path4)

path5 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(24)+"/angularStats_quat.csv"
data5 = pd.read_csv(path5)

path6 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(25)+"/angularStats_quat.csv"
data6 = pd.read_csv(path6)

In [10]:
qua_combined_1 = pd.concat([data1, data2, data3, data4, data5, data6], axis=0)

In [11]:
qua_combined_1.head()

,id,p_id,itr_id,prj_id,ppt_id,clt_id,hand,w,x,y,...,AVx,AVy,AVz,AAx,AAy,AAz,AJx,AJy,AJz,timestamp
0,212444,19,119,622,16,27,1,0.897418,0.059799,-0.405298,...,0.354371,-0.138088,-0.140597,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-13 11:24:11.316
1,212445,19,119,622,16,27,0,0.050747,0.424152,-0.057506,...,-0.477546,-0.222190,-4.942065,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-13 11:24:11.326
2,212446,19,119,622,16,27,1,0.899481,0.057231,-0.400058,...,-0.233459,0.107971,0.117765,-18.369691,7.689314,8.073829,-574.052834,240.291077,252.307168,2022-02-13 11:24:11.348
3,212447,19,119,622,16,27,0,0.091317,0.417244,-0.069474,...,-0.386306,-0.333672,-1.060516,2.943233,-3.596196,125.211266,94.943007,-116.006335,4039.073106,2022-02-13 11:24:11.357
4,212448,19,119,622,16,27,1,0.900094,0.055599,-0.397422,...,-0.152659,0.175423,0.231199,2.885740,2.409032,4.051195,759.122513,-188.581511,-143.665506,2022-02-13 11:24:11.376


In [13]:
write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/qua_combined_2022.csv"
qua_combined_1.to_csv(write_path, index = None)

